In [2]:
import pandas as pd

from Postgres_Controller import PostgresConnection
from IPython.display import clear_output

# Queries to know the data base

## Counting percentiles and samplings

In [2]:
query_n_percentiles = "SELECT COUNT(*) FROM percentiles;"
query_n_samplings = "SELECT COUNT(*) FROM percentil_groups;"

with PostgresConnection() as conn:
    cur = conn.cursor()

    print("Querying number of percentiles")
    %time cur.execute(query_n_percentiles)
    n_percentiles = cur.fetchone()[0]
    
    print("Querying number of samplings")
    %time cur.execute(query_n_samplings)
    n_samplings = cur.fetchone()[0]
    
    # close communication with the PostgreSQL database server
    cur.close()

Querying number of percentiles
CPU times: user 2.26 ms, sys: 1.19 ms, total: 3.46 ms
Wall time: 44.7 s
Querying number of samplings
CPU times: user 1.29 ms, sys: 1.37 ms, total: 2.66 ms
Wall time: 32 ms


In [3]:
print(f"We have {n_percentiles} percentiles and {n_samplings} samplings in the database.")

We have 20368177 percentiles and 923 samplings in the database.


In [4]:
query_n_percentiles = "SELECT * FROM percentiles;"

with PostgresConnection() as conn:
    cur = conn.cursor()

    print("Querying number of percentiles")
    %timeit -n 5 cur.execute(query_n_percentiles)
    
    # close communication with the PostgreSQL database server
    cur.close()

Querying number of percentiles
53.6 s ± 8.06 s per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [6]:
query_n_groups = "SELECT * FROM percentil_groups;"

with PostgresConnection() as conn:
    cur = conn.cursor()

    print("Querying number of percentiles")
    %timeit -n 5 cur.execute(query_n_groups)
    
    # close communication with the PostgreSQL database server
    cur.close()

Querying number of percentiles
11.4 ms ± 6.38 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [3]:
query = """
    SELECT 
        gene_name,
        percentile,
        project_id,
        metadata,
        number_genes,
        number_cells
    FROM 
        percentiles INNER JOIN percentil_groups ON percentiles.percentil_group = percentil_groups.id;
"""
with PostgresConnection() as conn:
    cur = conn.cursor()

    print("Querying")
    %timeit -n 5 cur.execute(query)
    
    # close communication with the PostgreSQL database server
    cur.close()

Querying
2min 55s ± 18.9 s per loop (mean ± std. dev. of 7 runs, 5 loops each)


## Metadata queries

In [4]:
query_n_projects = "SELECT COUNT(DISTINCT project_id) FROM percentil_groups;"

with PostgresConnection() as conn:
    cur = conn.cursor()

    print("Querying number of percentiles")
    %time cur.execute(query_n_projects)
    n_projects = cur.fetchone()[0]
    print(n_projects)
    # close communication with the PostgreSQL database server
    cur.close()

Querying number of percentiles
CPU times: user 464 µs, sys: 245 µs, total: 709 µs
Wall time: 30.4 ms
169


In [9]:
query = """
SELECT 
    project_id,
    SUM(number_genes) AS num_percentiles
FROM 
    percentil_groups
GROUP BY 
    project_id;
"""

with PostgresConnection() as conn:
    df = pd.read_sql_query(query, conn)

df['num_percentiles'].mean()

103257.68639053254

## Querying percentiles by gene_name

In [4]:
gene_name = "ENSG00000167468" # GPX4

query_percentile = f"""
    SELECT * 
    FROM percentiles
    WHERE
        gene_name = '{gene_name}';
"""

with PostgresConnection() as conn:
    cur = conn.cursor()

    print("Querying percentiles by gene name")
    %time cur.execute(query_percentile)
    percentiles = cur.fetchall()
    
    # close communication with the PostgreSQL database server
    cur.close()
    
percentiles[:5]

Querying percentiles by gene name
CPU times: user 1.79 ms, sys: 721 µs, total: 2.52 ms
Wall time: 52.6 ms


[(10846, 'ENSG00000167468', 98.54159621237741, 1),
 (34502, 'ENSG00000167468', 98.16482828380136, 2),
 (58158, 'ENSG00000167468', 96.70213329098513, 3),
 (81814, 'ENSG00000167468', 98.5434641432415, 4),
 (105470, 'ENSG00000167468', 98.97387216345719, 5)]

In [5]:
gene_name = "ENSG00000167468" # GPX4

query_percentile = f"""
    SELECT * 
    FROM percentiles
    WHERE
        gene_name = '{gene_name}';
"""

with PostgresConnection() as conn:
    %time percentiles = pd.read_sql_query(query_percentile, conn)
    
percentiles

CPU times: user 127 ms, sys: 97.1 ms, total: 224 ms
Wall time: 297 ms


,id,gene_name,percentile,percentil_group
0,10846,ENSG00000167468,98.541596,1
1,34502,ENSG00000167468,98.164828,2
2,58158,ENSG00000167468,96.702133,3
3,81814,ENSG00000167468,98.543464,4
4,105470,ENSG00000167468,98.973872,5
...,...,...,...,...
538,20276006,ENSG00000167468,95.508828,919
539,20296644,ENSG00000167468,96.830944,920
540,20317282,ENSG00000167468,97.083483,921
541,20337920,ENSG00000167468,95.525727,922


## Querying samplings by metadata

In [6]:
specie = "HomoSapiens"

query_sampling = f"""
    SELECT * 
    FROM percentil_groups
    WHERE
        metadata->>'organism' = '{specie}';
"""

with PostgresConnection() as conn:
    cur = conn.cursor()

    print("Querying samplings by specie")
    %time cur.execute(query_sampling)
    samplings = cur.fetchall()
    
    # close communication with the PostgreSQL database server
    cur.close()

samplings[:5]

Querying samplings by specie
CPU times: user 484 µs, sys: 456 µs, total: 940 µs
Wall time: 35.7 ms


[(1,
  '091cf39b-01bc-42e5-9437-f419a66c8a45',
  {'organism': 'HomoSapiens',
   'cell type': 'HematopoieticStemCell',
   'developmental stage': 'Adult',
   'disease': 'Control',
   'organism part': 'BoneMarrow'},
  23656,
  32445),
 (2,
  '091cf39b-01bc-42e5-9437-f419a66c8a45',
  {'organism': 'HomoSapiens',
   'developmental stage': 'Adult',
   'inferred cell type - ontology labels': 'CommonDendriticProgenitor'},
  20761,
  2075),
 (3,
  '091cf39b-01bc-42e5-9437-f419a66c8a45',
  {'organism': 'HomoSapiens',
   'developmental stage': 'Adult',
   'inferred cell type - ontology labels': 'CommonLymphoidProgenitor'},
  18891,
  3237),
 (4,
  '091cf39b-01bc-42e5-9437-f419a66c8a45',
  {'organism': 'HomoSapiens',
   'developmental stage': 'Adult',
   'inferred cell type - ontology labels': 'CommonMyeloidProgenitor'},
  21558,
  2328),
 (5,
  '091cf39b-01bc-42e5-9437-f419a66c8a45',
  {'organism': 'HomoSapiens',
   'developmental stage': 'Adult',
   'inferred cell type - ontology labels': 'Erythr

In [7]:
specie = "HomoSapiens"

query_sampling = f"""
    SELECT * 
    FROM percentil_groups
    WHERE
        metadata->>'organism' = '{specie}';
"""

with PostgresConnection() as conn:
     %time samplings = pd.read_sql_query(query_sampling, conn)
    
samplings

CPU times: user 6.59 ms, sys: 874 µs, total: 7.46 ms
Wall time: 40.8 ms


,id,project_id,metadata,number_genes,number_cells
0,1,091cf39b-01bc-42e5-9437-f419a66c8a45,"{'organism': 'HomoSapiens', 'cell type': 'Hema...",23656,32445
1,2,091cf39b-01bc-42e5-9437-f419a66c8a45,"{'organism': 'HomoSapiens', 'developmental sta...",20761,2075
2,3,091cf39b-01bc-42e5-9437-f419a66c8a45,"{'organism': 'HomoSapiens', 'developmental sta...",18891,3237
3,4,091cf39b-01bc-42e5-9437-f419a66c8a45,"{'organism': 'HomoSapiens', 'developmental sta...",21558,2328
4,5,091cf39b-01bc-42e5-9437-f419a66c8a45,"{'organism': 'HomoSapiens', 'developmental sta...",22122,3463
...,...,...,...,...,...
538,919,f8aa201c-4ff1-45a4-890e-840d63459ca2,"{'organism': 'HomoSapiens', 'developmental sta...",13649,63
539,920,f8aa201c-4ff1-45a4-890e-840d63459ca2,"{'organism': 'HomoSapiens', 'developmental sta...",16882,471
540,921,f8aa201c-4ff1-45a4-890e-840d63459ca2,"{'organism': 'HomoSapiens', 'developmental sta...",16698,539
541,922,f8aa201c-4ff1-45a4-890e-840d63459ca2,"{'organism': 'HomoSapiens', 'developmental sta...",12963,77


## Join between percentiles and samplings

In [8]:
gene_name = "ENSG00000167468" # GPX4

query_percentile = f"""
    SELECT 
        gene_name,
        percentile,
        project_id,
        metadata,
        number_genes,
        number_cells
    FROM 
        percentiles INNER JOIN percentil_groups ON percentiles.percentil_group = percentil_groups.id
    WHERE
        gene_name = '{gene_name}';
"""

with PostgresConnection() as conn:
    cur = conn.cursor()

    print("Querying percentiles by gene name")
    %time cur.execute(query_percentile)
    percentiles = cur.fetchall()
    
    # close communication with the PostgreSQL database server
    cur.close()
    
percentiles[:5]

Querying percentiles by gene name
CPU times: user 1.09 ms, sys: 685 µs, total: 1.78 ms
Wall time: 35.3 ms


[('ENSG00000167468',
  98.54159621237741,
  '091cf39b-01bc-42e5-9437-f419a66c8a45',
  {'organism': 'HomoSapiens',
   'cell type': 'HematopoieticStemCell',
   'developmental stage': 'Adult',
   'disease': 'Control',
   'organism part': 'BoneMarrow'},
  23656,
  32445),
 ('ENSG00000167468',
  98.16482828380136,
  '091cf39b-01bc-42e5-9437-f419a66c8a45',
  {'organism': 'HomoSapiens',
   'developmental stage': 'Adult',
   'inferred cell type - ontology labels': 'CommonDendriticProgenitor'},
  20761,
  2075),
 ('ENSG00000167468',
  96.70213329098513,
  '091cf39b-01bc-42e5-9437-f419a66c8a45',
  {'organism': 'HomoSapiens',
   'developmental stage': 'Adult',
   'inferred cell type - ontology labels': 'CommonLymphoidProgenitor'},
  18891,
  3237),
 ('ENSG00000167468',
  98.5434641432415,
  '091cf39b-01bc-42e5-9437-f419a66c8a45',
  {'organism': 'HomoSapiens',
   'developmental stage': 'Adult',
   'inferred cell type - ontology labels': 'CommonMyeloidProgenitor'},
  21558,
  2328),
 ('ENSG00000167

In [9]:
gene_name = "ENSG00000167468" # GPX4

query_percentile = f"""
    SELECT 
        gene_name,
        percentile,
        project_id,
        metadata,
        number_genes,
        number_cells
    FROM 
        percentiles INNER JOIN percentil_groups ON percentiles.percentil_group = percentil_groups.id
    WHERE
        gene_name = '{gene_name}';
"""

with PostgresConnection() as conn:
    %time percentiles = pd.read_sql_query(query_percentile, conn)
    
percentiles

CPU times: user 14.1 ms, sys: 1.5 ms, total: 15.6 ms
Wall time: 75.4 ms


,gene_name,percentile,project_id,metadata,number_genes,number_cells
0,ENSG00000167468,98.541596,091cf39b-01bc-42e5-9437-f419a66c8a45,"{'organism': 'HomoSapiens', 'cell type': 'Hema...",23656,32445
1,ENSG00000167468,98.164828,091cf39b-01bc-42e5-9437-f419a66c8a45,"{'organism': 'HomoSapiens', 'developmental sta...",20761,2075
2,ENSG00000167468,96.702133,091cf39b-01bc-42e5-9437-f419a66c8a45,"{'organism': 'HomoSapiens', 'developmental sta...",18891,3237
3,ENSG00000167468,98.543464,091cf39b-01bc-42e5-9437-f419a66c8a45,"{'organism': 'HomoSapiens', 'developmental sta...",21558,2328
4,ENSG00000167468,98.973872,091cf39b-01bc-42e5-9437-f419a66c8a45,"{'organism': 'HomoSapiens', 'developmental sta...",22122,3463
...,...,...,...,...,...,...
538,ENSG00000167468,95.508828,f8aa201c-4ff1-45a4-890e-840d63459ca2,"{'organism': 'HomoSapiens', 'developmental sta...",13649,63
539,ENSG00000167468,96.830944,f8aa201c-4ff1-45a4-890e-840d63459ca2,"{'organism': 'HomoSapiens', 'developmental sta...",16882,471
540,ENSG00000167468,97.083483,f8aa201c-4ff1-45a4-890e-840d63459ca2,"{'organism': 'HomoSapiens', 'developmental sta...",16698,539
541,ENSG00000167468,95.525727,f8aa201c-4ff1-45a4-890e-840d63459ca2,"{'organism': 'HomoSapiens', 'developmental sta...",12963,77


## Counting genes

In [10]:
gene_name = "ENSG00000167468" # GPX4

query_percentile = f"""
    SELECT 
        gene_name,
        COUNT (*) AS num_percentiles
    FROM 
        percentiles
    GROUP BY
        gene_name;
"""

with PostgresConnection() as conn:
    %time percentiles = pd.read_sql_query(query_percentile, conn)
    
percentiles

CPU times: user 186 ms, sys: 62 ms, total: 248 ms
Wall time: 5.24 s


,gene_name,num_percentiles
0,AGAP000002,3
1,AGAP000005,5
2,AGAP000007,5
3,AGAP000008,5
4,AGAP000009,5
...,...,...
226438,YPR201W,2
226439,YPR202W,2
226440,YPR203W,2
226441,YPR204C-A,2


In [11]:
print(f"In our database we have {len(percentiles)} different genes, with a mean of {percentiles['num_percentiles'].mean():.2f} percentiles per gene.")

In our database we have 226443 different genes, with a mean of 89.95 percentiles per gene.


In [12]:
specie = "HomoSapiens"

query_percentile = f"""
    SELECT 
        gene_name,
        COUNT (*) AS num_percentiles
    FROM 
        percentiles INNER JOIN percentil_groups ON percentiles.percentil_group = percentil_groups.id
    WHERE
        metadata->>'organism' = '{specie}'
    GROUP BY
        gene_name;
"""

with PostgresConnection() as conn:
    %time percentiles = pd.read_sql_query(query_percentile, conn)
    
percentiles

CPU times: user 32.2 ms, sys: 5.05 ms, total: 37.3 ms
Wall time: 7.6 s


,gene_name,num_percentiles
0,ENSG00000079805,543
1,ENSG00000164611,543
2,ENSG00000219993,447
3,ENSG00000035115,543
4,ENSG00000215452,482
...,...,...
35003,ENSG00000230376,55
35004,ENSG00000261151,101
35005,ENSG00000250625,14
35006,ENSG00000204705,136


In [13]:
print(f"In our database we have {len(percentiles)} different human genes, with a mean of {percentiles['num_percentiles'].mean():.2f} percentiles per gene.")

In our database we have 35008 different human genes, with a mean of 362.90 percentiles per gene.


In [14]:
specie = "HomoSapiens"

query_percentile = f"""
    SELECT 
        project_id,
        COUNT (*) AS num_percentiles
    FROM 
        percentiles INNER JOIN percentil_groups ON percentiles.percentil_group = percentil_groups.id
    GROUP BY
        project_id;
"""

with PostgresConnection() as conn:
    %time percentiles = pd.read_sql_query(query_percentile, conn)
    
percentiles

CPU times: user 2.06 ms, sys: 238 µs, total: 2.3 ms
Wall time: 5.44 s


,project_id,num_percentiles
0,091cf39b-01bc-42e5-9437-f419a66c8a45,212904
1,116965f3-f094-4769-9d28-ae675c1b569c,131005
2,2043c65a-1cf8-4828-a656-9e247d4e64f1,217096
3,4a95101c-9ffc-4f30-a809-f04518a23803,182056
4,4d6f6c96-2a83-43d8-8fe1-0f53bffd4674,286155
...,...,...
164,E-MTAB-8559,43450
165,E-MTAB-8698,35944
166,E-MTAB-8810,382266
167,E-MTAB-9221,151664


In [15]:
print(f"In our database we have {len(percentiles)} different projects, with a mean of {percentiles['num_percentiles'].mean():.2f} percentiles per project.")

In our database we have 169 different projects, with a mean of 120521.76 percentiles per project.


## Metadata information

### Organisms

In [15]:
query_percentile = f"""
    SELECT 
        metadata->>'organism' AS organism,
        COUNT(*) AS num_samplings
    FROM 
        percentil_groups
    GROUP BY
        metadata->>'organism';
"""

with PostgresConnection() as conn:
    %time organism_samplings = pd.read_sql_query(query_percentile, conn)
    
organism_samplings

CPU times: user 1.51 ms, sys: 0 ns, total: 1.51 ms
Wall time: 124 ms


,organism,num_samplings
0,AnophelesGambiae,5
1,DanioRerio,23
2,MusMusculus,242
3,PlasmodiumFalciparum,2
4,GallusGallus,45
5,CaenorhabditisElegans,2
6,PlasmodiumBerghei,2
7,SchistosomaMansoni,2
8,CallithrixJacchus,6
9,DrosophilaMelanogaster,28


In [16]:
query_percentile = f"""
    SELECT
        metadata->>'organism' AS organism,
        COUNT(DISTINCT project_id) AS num_projects
    FROM
        percentil_groups
    GROUP BY
        metadata->>'organism';
"""

with PostgresConnection() as conn:
    %time organism_projects = pd.read_sql_query(query_percentile, conn)
    
organism_projects

CPU times: user 881 µs, sys: 0 ns, total: 881 µs
Wall time: 124 ms


,organism,num_projects
0,AnophelesGambiae,2
1,ArabidopsisThaliana,5
2,CaenorhabditisElegans,1
3,CallithrixJacchus,1
4,DanioRerio,7
5,DrosophilaMelanogaster,7
6,GallusGallus,4
7,HomoSapiens,77
8,MusMusculus,59
9,PlasmodiumBerghei,2


In [19]:
query_percentile = f"""
    SELECT
        metadata->>'organism' AS organism,
        COUNT(DISTINCT gene_name) AS num_genes
    FROM
        percentiles INNER JOIN percentil_groups ON percentiles.percentil_group = percentil_groups.id
    GROUP BY
        metadata->>'organism';
"""

with PostgresConnection() as conn:
    %time organism_genes = pd.read_sql_query(query_percentile, conn)
    
organism_genes

CPU times: user 12.4 ms, sys: 67 µs, total: 12.5 ms
Wall time: 7min 13s


,organism,num_genes
0,AnophelesGambiae,8071
1,ArabidopsisThaliana,23848
2,CaenorhabditisElegans,12139
3,CallithrixJacchus,20532
4,DanioRerio,24231
5,DrosophilaMelanogaster,13572
6,GallusGallus,16005
7,HomoSapiens,35008
8,MusMusculus,35478
9,PlasmodiumBerghei,4893


In [2]:
query_percentile = f"""
    SELECT
        metadata->>'organism' AS organism,
        COUNT(gene_name) AS num_percentiles
    FROM
        percentiles INNER JOIN percentil_groups ON percentiles.percentil_group = percentil_groups.id
    GROUP BY
        metadata->>'organism';
"""

with PostgresConnection() as conn:
    %time organism_percentiles = pd.read_sql_query(query_percentile, conn)
    
organism_percentiles

CPU times: user 1.56 ms, sys: 1.51 ms, total: 3.07 ms
Wall time: 25.4 s


,organism,num_percentiles
0,AnophelesGambiae,31100
1,ArabidopsisThaliana,400995
2,CaenorhabditisElegans,24278
3,CallithrixJacchus,123192
4,DanioRerio,430436
5,DrosophilaMelanogaster,315306
6,GallusGallus,642561
7,HomoSapiens,12704304
8,MusMusculus,5620903
9,PlasmodiumBerghei,9770


In [17]:
query_percentile = f"""
    SELECT
        metadata->>'organism' AS organism,
        SUM(number_genes) AS num_percentiles_expressed
    FROM
        percentil_groups
    GROUP BY
        metadata->>'organism';
"""

with PostgresConnection() as conn:
    %time organism_percentiles_expressed = pd.read_sql_query(query_percentile, conn)
    
organism_percentiles_expressed

CPU times: user 502 µs, sys: 718 µs, total: 1.22 ms
Wall time: 120 ms


,organism,num_percentiles_expressed
0,AnophelesGambiae,30049
1,DanioRerio,384373
2,MusMusculus,4908507
3,PlasmodiumFalciparum,10239
4,GallusGallus,561731
5,CaenorhabditisElegans,22745
6,PlasmodiumBerghei,9770
7,SchistosomaMansoni,13618
8,CallithrixJacchus,112960
9,DrosophilaMelanogaster,252825


In [25]:
organism_projects.merge(organism_samplings, on="organism")\
                 .merge(organism_genes, on="organism")\
                 .merge(organism_percentiles, on="organism")\
                 .merge(organism_percentiles_expressed, on="organism")

,organism,num_projects,num_samplings,num_genes,num_percentiles,num_percentiles_expressed
0,AnophelesGambiae,2,5,8071,31100,30049
1,ArabidopsisThaliana,5,19,23848,400995,349495
2,CaenorhabditisElegans,1,2,12139,24278,22745
3,CallithrixJacchus,1,6,20532,123192,112960
4,DanioRerio,7,23,24231,430436,384373
5,DrosophilaMelanogaster,7,28,13572,315306,252825
6,GallusGallus,4,45,16005,642561,561731
7,HomoSapiens,77,543,35008,12704304,10753573
8,MusMusculus,59,242,35478,5620903,4908507
9,PlasmodiumBerghei,2,2,4893,9770,9770


In [2]:
def create_metadata_info_table(metadata_name, column_name="metadata_name"):
    query_sampling = f"""
        SELECT 
            metadata->>'{metadata_name}' AS {column_name},
            COUNT(*) AS num_samplings
        FROM 
            percentil_groups
        GROUP BY
            metadata->>'{metadata_name}';
    """

    query_projects = f"""
        SELECT
            metadata->>'{metadata_name}' AS {column_name},
            COUNT(DISTINCT project_id) AS num_projects
        FROM
            percentil_groups
        GROUP BY
            metadata->>'{metadata_name}';
    """
    
    query_genes = f"""
        SELECT
            metadata->>'{metadata_name}' AS {column_name},
            COUNT(DISTINCT gene_name) AS num_genes
        FROM
            percentiles INNER JOIN percentil_groups ON percentiles.percentil_group = percentil_groups.id
        GROUP BY
            metadata->>'{metadata_name}';
    """
    
    query_percentile = f"""
        SELECT
            metadata->>'{metadata_name}' AS {column_name},
            COUNT(gene_name) AS num_percentiles
        FROM
            percentiles INNER JOIN percentil_groups ON percentiles.percentil_group = percentil_groups.id
        GROUP BY
            metadata->>'{metadata_name}';
    """

    query_percentile_expressed = f"""
        SELECT
            metadata->>'{metadata_name}' AS {column_name},
            SUM(number_genes) AS num_percentiles_expressed
        FROM
            percentil_groups
        GROUP BY
            metadata->>'{metadata_name}';
    """
    
    with PostgresConnection() as conn:
        print("Getting projects")
        projects = pd.read_sql_query(query_sampling, conn)
        print("Getting samplings")
        samplings = pd.read_sql_query(query_projects, conn)
        print("Getting genes")
        genes = pd.read_sql_query(query_genes, conn)
        print("Getting percentiles")
        percentiles = pd.read_sql_query(query_percentile, conn)
        print("Getting percentiles_expressed")
        percentiles_expressed = pd.read_sql_query(query_percentile_expressed, conn)
        
        df = projects.merge(samplings, on=column_name)\
                     .merge(genes, on=column_name)\
                     .merge(percentiles, on=column_name)\
                     .merge(percentiles_expressed, on=column_name)

    return df

In [3]:
query = f"""
    SELECT
        DISTINCT json_object_keys(metadata)
    FROM
        percentil_groups;
"""

with PostgresConnection() as conn:
    %time organism_genes = pd.read_sql_query(query, conn)
    
organism_genes

CPU times: user 1.51 ms, sys: 0 ns, total: 1.51 ms
Wall time: 224 ms


,json_object_keys
0,disease
1,developmental stage
2,organism
3,inferred cell type - ontology labels
4,organism part
5,cell type
6,sampling site
7,metastatic site
8,biopsy site


In [4]:
metadata_names = [
    'organism',
    'cell type',
    'developmental stage',
    'inferred cell type - ontology labels',
    'disease',
    'organism part',
    'sampling site',
    'biopsy site',
    'metastatic site'
]  

column_names = [
    'organism',
    'cell_type',
    'developmental_stage',
    'inferred_cell_type',
    'disease',
    'organism_part',
    'sampling_site',
    'biopsy_site',
    'metastatic_site'
] 

In [6]:
dataframes = []

for metadata_name in metadata_names:
    clear_output(wait=True)
    print("Getting info from " + metadata_name)
    dataframes.append(create_metadata_info_table(metadata_name))
    
pd.concat(dataframes, keys=column_names).to_csv("metadata_info.csv", sep="\t")

Getting info from metastatic site
Getting projects
Getting samplings
Getting genes
Getting percentiles
Getting percentiles_expressed


NameError: name 'columns_names' is not defined

In [17]:
metadata_type = [len(df) * [column_name] for column_name, df in zip(column_names, dataframes)]
metadata_type = [item for sublist in metadata_type for item in sublist]
metadata_type

['organism',
 'organism',
 'organism',
 'organism',
 'organism',
 'organism',
 'organism',
 'organism',
 'organism',
 'organism',
 'organism',
 'organism',
 'organism',
 'organism',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell_type',
 'cell

In [22]:
df = pd.concat(dataframes).reset_index(drop=True)
df['metadata_type'] = metadata_type
df

,metadata_name,num_samplings,num_projects,num_genes,num_percentiles,num_percentiles_expressed,metadata_type
0,AnophelesGambiae,5,2,8071,31100,30049,organism
1,DanioRerio,23,7,24231,430436,384373,organism
2,MusMusculus,242,59,35478,5620903,4908507,organism
3,PlasmodiumFalciparum,2,1,5189,10378,10239,organism
4,GallusGallus,45,4,16005,642561,561731,organism
...,...,...,...,...,...,...,...
570,Lung,1,1,20211,20211,20073,metastatic_site
571,LymphNode,3,2,26865,68261,66984,metastatic_site
572,SubcutaneousMetastasis,1,1,21852,21852,21852,metastatic_site
573,Alveolus,2,1,22512,45024,42360,metastatic_site


In [23]:
df.to_csv('metadata_info.csv')

In [28]:
query_percentile = f"""
    SELECT
        metadata->>'organism' AS organism,
        COUNT(DISTINCT gene_name) AS num_genes
    FROM
        percentiles INNER JOIN percentil_groups ON percentiles.percentil_group = percentil_groups.id
    GROUP BY
        metadata->>'organism';
"""

with PostgresConnection() as conn:
    cur = conn.cursor()

    print("Querying percentiles by gene name")
    %time cur.execute(query_percentile)
    percentiles = cur.fetchall()
    
    # close communication with the PostgreSQL database server
    cur.close()
    
percentiles[:5]

Querying percentiles by gene name
CPU times: user 8.65 ms, sys: 178 µs, total: 8.83 ms
Wall time: 6min 27s


[('AnophelesGambiae', 8071),
 ('ArabidopsisThaliana', 23848),
 ('CaenorhabditisElegans', 12139),
 ('CallithrixJacchus', 20532),
 ('DanioRerio', 24231)]

# Application queries examples

We are going to design some queries and check the time they take.

First, we will drop the index and will see the time without it so we can then create the index and see how they improve the expected time by query.

In [15]:
command = '''
    DROP INDEX IF EXISTS index_percentiles_id;
    DROP INDEX IF EXISTS index_percentiles_gene_name;
    DROP INDEX IF EXISTS index_percentiles_percentil_group;
    
    DROP INDEX IF EXISTS index_percentil_groups_id;
    DROP INDEX IF EXISTS index_percentil_groups_project_id;
    DROP INDEX IF EXISTS index_percentil_groups_metadata_cell_type;   
    DROP INDEX IF EXISTS index_percentil_groups_metadata_organism;   
'''

with PostgresConnection() as conn:
        cur = conn.cursor()
        # execute command
        %time cur.execute(command)
        # close communication with the PostgreSQL database server
        cur.close()
        # commit the changes
        conn.commit()


CPU times: user 246 µs, sys: 109 µs, total: 355 µs
Wall time: 682 ms


## Application queries examples without index

In this first query, we are looking for a concrete gene for Parkinson's Disease in human.

In [17]:
gene_name = "ENSG00000167468" # GPX4
disease = 'ParkinsonsDisease'
specie = 'HomoSapiens'

query_percentile = f"""
    SELECT 
        gene_name,
        percentile,
        project_id,
        metadata,
        metadata->'disease' AS disease,
        metadata->'cell type' AS cell_type,
        number_genes,
        number_cells
    FROM 
        percentiles INNER JOIN percentil_groups ON percentiles.percentil_group = percentil_groups.id
    WHERE
        gene_name = '{gene_name}' AND
        metadata->>'organism' = '{specie}' AND
        metadata->>'disease' = '{disease}';
"""

with PostgresConnection() as conn:
    %time percentiles = pd.read_sql_query(query_percentile, conn)
    
percentiles

CPU times: user 2.01 ms, sys: 122 µs, total: 2.13 ms
Wall time: 3.09 s


,gene_name,percentile,project_id,metadata,disease,cell_type,number_genes,number_cells
0,ENSG00000167468,98.659703,E-MTAB-7303,"{'organism': 'HomoSapiens', 'cell type': 'Dopa...",ParkinsonsDisease,DopaminergicNeuron,15892,37
1,ENSG00000167468,98.659703,E-MTAB-7303,"{'organism': 'HomoSapiens', 'developmental sta...",ParkinsonsDisease,None,15892,37


Now, we want to search the same gene in human, but for Dopaminergic Neurons.

In [18]:
gene_name = "ENSG00000167468" # GPX4
cell_type = 'DopaminergicNeuron'
specie = 'HomoSapiens'

query_percentile = f"""
    SELECT 
        gene_name,
        percentile,
        project_id,
        metadata,
        metadata->'disease' AS disease,
        metadata->'cell type' AS cell_type,
        number_genes,
        number_cells
    FROM 
        percentiles INNER JOIN percentil_groups ON percentiles.percentil_group = percentil_groups.id
    WHERE
        gene_name = '{gene_name}' AND
        metadata->>'organism' = '{specie}' AND
        metadata->>'cell type' = '{cell_type}';
"""

with PostgresConnection() as conn:
    %time percentiles = pd.read_sql_query(query_percentile, conn)
    
percentiles

CPU times: user 2.32 ms, sys: 166 µs, total: 2.49 ms
Wall time: 3.15 s


,gene_name,percentile,project_id,metadata,disease,cell_type,number_genes,number_cells
0,ENSG00000167468,98.659703,E-MTAB-7303,"{'organism': 'HomoSapiens', 'cell type': 'Dopa...",ParkinsonsDisease,DopaminergicNeuron,15892,37
1,ENSG00000167468,99.178163,E-MTAB-7303,"{'organism': 'HomoSapiens', 'cell type': 'Dopa...",Control,DopaminergicNeuron,17035,86


Another type of query we can do is search genes for a concrete project.

In [19]:
project_id = "E-CURD-55"

query_percentile = f"""
    SELECT 
        gene_name,
        percentile,
        project_id,
        metadata,
        number_genes,
        number_cells
    FROM 
        percentiles INNER JOIN percentil_groups ON percentiles.percentil_group = percentil_groups.id
    WHERE
        project_id = '{project_id}';
"""

with PostgresConnection() as conn:
    %time percentiles = pd.read_sql_query(query_percentile, conn)
    
percentiles

CPU times: user 312 ms, sys: 35.2 ms, total: 347 ms
Wall time: 3.14 s


,gene_name,percentile,project_id,metadata,number_genes,number_cells
0,ENSG00000065150,78.076942,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",20873,74355
1,ENSG00000000003,30.307095,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",20873,74355
2,ENSG00000000419,69.510851,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",20873,74355
3,ENSG00000000457,60.312365,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",20873,74355
4,ENSG00000000460,50.079049,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",20873,74355
...,...,...,...,...,...,...
41787,ENSG00000229644,38.748539,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",19673,23233
41788,ENSG00000133703,91.994104,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",19673,23233
41789,ENSG00000133704,70.467138,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",19673,23233
41790,ENSG00000133706,85.579220,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",19673,23233


Or get a subgroup of percentiles for a project.

In [20]:
project_id = "E-CURD-55"

query_percentile = f"""
    SELECT 
        gene_name,
        percentile,
        project_id,
        metadata,
        number_genes,
        number_cells
    FROM 
        percentiles INNER JOIN percentil_groups ON percentiles.percentil_group = percentil_groups.id
    WHERE
        project_id = '{project_id}' AND
        percentile > 90;
"""

with PostgresConnection() as conn:
    %time percentiles = pd.read_sql_query(query_percentile, conn)
    
percentiles

CPU times: user 24.3 ms, sys: 1.71 ms, total: 26 ms
Wall time: 1.74 s


,gene_name,percentile,project_id,metadata,number_genes,number_cells
0,ENSG00000000938,97.365017,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",20873,74355
1,ENSG00000006652,95.822354,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",20873,74355
2,ENSG00000007080,90.365544,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",20873,74355
3,ENSG00000007168,92.564557,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",20873,74355
4,ENSG00000007264,91.294974,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",20873,74355
...,...,...,...,...,...,...
4051,ENSG00000157191,90.087938,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",19673,23233
4052,ENSG00000176731,95.572612,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",19673,23233
4053,ENSG00000176978,96.594317,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",19673,23233
4054,ENSG00000177105,97.799014,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",19673,23233


Or even get the number of groups (samplings) a project has.

In [21]:
project_id = "E-CURD-55"

query_percentile = f"""
    SELECT 
        project_id,
        COUNT(*)
    FROM 
        percentil_groups
    WHERE
        project_id = '{project_id}'
    GROUP BY
        project_id;

"""

with PostgresConnection() as conn:
    %time percentiles = pd.read_sql_query(query_percentile, conn)
    
percentiles

CPU times: user 2.99 ms, sys: 254 µs, total: 3.25 ms
Wall time: 49.3 ms


,project_id,count
0,E-CURD-55,2


## Creating index

We can now create the index of the columns we are using in the queries.

In [16]:
command = '''
    CREATE INDEX IF NOT EXISTS index_percentiles_id ON percentiles(id);
    CREATE INDEX IF NOT EXISTS index_percentiles_gene_name ON percentiles(gene_name);
    CREATE INDEX IF NOT EXISTS index_percentiles_percentil_group ON percentiles(percentil_group);
    
    CREATE INDEX IF NOT EXISTS index_percentil_groups_id ON percentil_groups(id);
    CREATE INDEX IF NOT EXISTS index_percentil_groups_project_id ON percentil_groups(project_id);
    CREATE INDEX IF NOT EXISTS index_percentil_groups_metadata_cell_type ON percentil_groups((metadata ->> 'cell type'));    
    CREATE INDEX IF NOT EXISTS index_percentil_groups_metadata_organism ON percentil_groups((metadata ->> 'organism'));    
'''

with PostgresConnection() as conn:
        cur = conn.cursor()
        # execute command
        %time cur.execute(command)
        # close communication with the PostgreSQL database server
        cur.close()
        # commit the changes
        conn.commit()


CPU times: user 25.3 ms, sys: 13.6 ms, total: 38.9 ms
Wall time: 17min 21s


## Application queries examples with index

Finally, we repeat all the previous queries and we can appreciate a clear improvement.

In [24]:
gene_name = "ENSG00000167468" # GPX4
disease = 'ParkinsonsDisease'
specie = 'HomoSapiens'

query_percentile = f"""
    SELECT 
        gene_name,
        percentile,
        project_id,
        metadata,
        metadata->'disease' AS disease,
        metadata->'cell type' AS cell_type,
        number_genes,
        number_cells
    FROM 
        percentiles INNER JOIN percentil_groups ON percentiles.percentil_group = percentil_groups.id
    WHERE
        gene_name = '{gene_name}' AND
        metadata->>'organism' = '{specie}' AND
        metadata->>'disease' = '{disease}';
"""

with PostgresConnection() as conn:
    %time percentiles = pd.read_sql_query(query_percentile, conn)
    
percentiles

CPU times: user 2.54 ms, sys: 0 ns, total: 2.54 ms
Wall time: 239 ms


,gene_name,percentile,project_id,metadata,disease,cell_type,number_genes,number_cells
0,ENSG00000167468,98.659703,E-MTAB-7303,"{'organism': 'HomoSapiens', 'cell type': 'Dopa...",ParkinsonsDisease,DopaminergicNeuron,15892,37
1,ENSG00000167468,98.659703,E-MTAB-7303,"{'organism': 'HomoSapiens', 'developmental sta...",ParkinsonsDisease,None,15892,37


In [25]:
gene_name = "ENSG00000167468" # GPX4
cell_type = 'DopaminergicNeuron'
specie = 'HomoSapiens'

query_percentile = f"""
    SELECT 
        gene_name,
        percentile,
        project_id,
        metadata,
        metadata->'disease' AS disease,
        metadata->'cell type' AS cell_type,
        number_genes,
        number_cells
    FROM 
        percentiles INNER JOIN percentil_groups ON percentiles.percentil_group = percentil_groups.id
    WHERE
        gene_name = '{gene_name}' AND
        metadata->>'organism' = '{specie}' AND
        metadata->>'cell type' = '{cell_type}';
"""

with PostgresConnection() as conn:
    %time percentiles = pd.read_sql_query(query_percentile, conn)
    
percentiles

CPU times: user 2.53 ms, sys: 0 ns, total: 2.53 ms
Wall time: 161 ms


,gene_name,percentile,project_id,metadata,disease,cell_type,number_genes,number_cells
0,ENSG00000167468,98.659703,E-MTAB-7303,"{'organism': 'HomoSapiens', 'cell type': 'Dopa...",ParkinsonsDisease,DopaminergicNeuron,15892,37
1,ENSG00000167468,99.178163,E-MTAB-7303,"{'organism': 'HomoSapiens', 'cell type': 'Dopa...",Control,DopaminergicNeuron,17035,86


In [26]:
project_id = "E-CURD-55"

query_percentile = f"""
    SELECT 
        gene_name,
        percentile,
        project_id,
        metadata,
        number_genes,
        number_cells
    FROM 
        percentiles INNER JOIN percentil_groups ON percentiles.percentil_group = percentil_groups.id
    WHERE
        project_id = '{project_id}';
"""

with PostgresConnection() as conn:
    %time percentiles = pd.read_sql_query(query_percentile, conn)
    
percentiles

CPU times: user 197 ms, sys: 20.7 ms, total: 218 ms
Wall time: 2.04 s


,gene_name,percentile,project_id,metadata,number_genes,number_cells
0,ENSG00000000003,30.307095,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",20873,74355
1,ENSG00000000419,69.510851,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",20873,74355
2,ENSG00000000457,60.312365,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",20873,74355
3,ENSG00000000460,50.079049,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",20873,74355
4,ENSG00000000938,97.365017,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",20873,74355
...,...,...,...,...,...,...
41787,ENSG00000288520,45.188837,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",19673,23233
41788,ENSG00000288534,49.184161,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",19673,23233
41789,ENSG00000288550,47.623647,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",19673,23233
41790,ENSG00000288558,66.121080,E-CURD-55,"{'organism': 'HomoSapiens', 'developmental sta...",19673,23233


In [27]:
project_id = "E-CURD-55"

query_percentile = f"""
    SELECT 
        project_id,
        COUNT(*)
    FROM 
        percentil_groups
    WHERE
        project_id = '{project_id}'
    GROUP BY
        project_id;

"""

with PostgresConnection() as conn:
    %time percentiles = pd.read_sql_query(query_percentile, conn)
    
percentiles

CPU times: user 996 µs, sys: 505 µs, total: 1.5 ms
Wall time: 139 ms


,project_id,count
0,E-CURD-55,2
